# 0. Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))
import ss_api_call as ss

📂 Current working directory: /Users/serenekim/Desktop/PhD/meta-wealth_mobility
💚 Today is 2025-10-31


# 1. Triangles

In [2]:
tri = pd.read_csv(f"{path}/results/feature-only-KG/triangle_counts_papers.csv")
len(tri)

495

In [15]:
import re, unicodedata
import pandas as pd

# --- 1) Normalizer: unify Unicode dashes to "-" and collapse spaces ---
def norm(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"[\u2010\u2011\u2012\u2013\u2014\u2212]", "-", s)  # dash variants → "-"
    s = re.sub(r"\s+", " ", s).strip()
    return s

for base in ("m","dt","rq"):
    tri[f"{base}.name_norm"] = tri[f"{base}.name"].map(norm)

# --- 2) Your label → code dictionaries (raw), then normalized once ---
M_MAP = {
    "Regression‐based Measures": 1,
    "Rank‐based Measures": 2,
    "Transition Matrix / Probability Measures": 3,
    "Absolute Mobility Measures": 4,
    "Multigenerational Measures": 5,
    "Decomposition / Structural Approaches": 6,
    "Non-parametric Approaches": 7,
    "Others_Measure": 8,
}
DT_MAP = {
    "Panel/Longitudinal Surveys": 1,
    "Administrative/Registry Data": 2,
    "National Survey Data": 3,
    "Opportunity Atlas": 4,
    "Natural/Experimental Data": 5,
    "Linked Administrative Data": 6,
    "International Panel Data": 7,
    "Rich List Data": 8,
    "University/Institution Data": 9,
    "Pseudo-Panel/Household Budget Survey": 10,
    "Archival/Historical Data": 11,
    "Big Data": 12,
    "No dataset": 13,
    "Others_DataType": 14,
}
RQ_MAP = {
    "Measurement and Methodological Advances": 1,
    "Empirical Estimates and Determinants": 2,
    "Policy, Institutional, and Geographic Impacts": 3,
    "Intergenerational Wealth Mobility and Inheritance": 4,
    "Demographic Differences in Mobility (Race, Gender, etc.)": 5,
    "Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)": 6,
    "Theoretical and Structural Models": 7,
    "Perceptions of Mobility and Attitudes": 8,
    "Others_RqType": 9,
}

M_MAP = {norm(k): v for k,v in M_MAP.items()}
DT_MAP = {norm(k): v for k,v in DT_MAP.items()}
RQ_MAP = {norm(k): v for k,v in RQ_MAP.items()}

# --- 3) Vectorized mapping (nullable Int64 keeps NA if truly unseen) ---
tri["m.code"]  = tri["m.name_norm"].map(M_MAP).astype("Int64")
tri["dt.code"] = tri["dt.name_norm"].map(DT_MAP).astype("Int64")
tri["rq.code"] = tri["rq.name_norm"].map(RQ_MAP).astype("Int64")

tri["code_combined"] = tri[['m.code', 'dt.code', 'rq.code']].apply(lambda x: [v for v in x if pd.notna(v)], axis=1)
tri["code_combined_flt"] = tri[['m.code', 'dt.code', 'rq.code']].apply(lambda x: [v for v in x if pd.notna(v)], axis=1)
tri['code_combined'] = tri['code_combined'].astype('string')
# (optional) quick diagnostics for any still-unmapped labels
for base in ("m","dt","rq"):
    miss = tri.loc[tri[f"{base}.code"].isna(), f"{base}.name"].unique()
    if len(miss): print(f"Unmapped {base}.name values:", miss)


In [16]:
tri['names'].nunique(), tri['code_combined'].nunique() 

(227, 227)

In [17]:
tri['period2'] = tri['period'].apply(lambda x: x.split('-')[1].strip())
export = tri[['period2', 'names', 'n_papers', 'code_combined']].copy()
# export.to_csv(f"{path}/results/feature-only-KG/triangle_counts_papers_p2.csv", index=False)
export

,period2,names,n_papers,code_combined
0,2000,"[No dataset, Decomposition / Structural Approa...",4,"[6, 13, 7]"
1,2000,"[Administrative/Registry Data, Regression‐base...",2,"[1, 2, 4]"
2,2000,"[Linked Administrative Data, Regression‐based ...",2,"[1, 6, 4]"
3,2000,"[No dataset, Regression‐based Measures, Empiri...",2,"[1, 13, 2]"
4,2000,"[Panel/Longitudinal Surveys, Regression‐based ...",2,"[1, 1, 2]"
...,...,...,...,...
490,2025,"[Panel/Longitudinal Surveys, Regression‐based ...",0,"[1, 1, 6]"
491,2025,"[Opportunity Atlas, Regression‐based Measures,...",0,"[1, 4, 6]"
492,2025,"[Others_DataType, Rank‐based Measures, Mobilit...",0,"[2, 14, 6]"
493,2025,"[Panel/Longitudinal Surveys, Rank‐based Measur...",0,"[2, 1, 6]"


In [18]:
first = export.groupby('period2').first()
np.column_stack([first.index, first['names'], first['code_combined']])  # returns a numpy array with period2 and names

array([['2000',
        '[No dataset, Decomposition / Structural Approaches, Theoretical and Structural Models]',
        '[6, 13, 7]'],
       ['2005',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2010',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2015',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2020',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]',
        '[1, 1, 2]'],
       ['2025',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]',
        '[1, 1, 2]']], dtype=object)

In [19]:
export.groupby(['names', 'code_combined']).sum('n_papers').sort_values('n_papers', ascending=False).head(11)

,,n_papers
names,code_combined,
"[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 1, 2]",47
"[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 1, 4]",47
"[Linked Administrative Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 6, 4]",30
"[National Survey Data, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 3, 2]",24
"[National Survey Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 3, 4]",23
"[No dataset, Decomposition / Structural Approaches, Theoretical and Structural Models]","[6, 13, 7]",21
"[No dataset, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 13, 2]",19
"[Administrative/Registry Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 2, 4]",15
"[Administrative/Registry Data, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 2, 2]",13


## Beeswarm plot with Plotly (I don't like it)

In [20]:
# df = tri[tri['n_papers']>0].copy()
df = tri.copy()
len(df)

495

In [21]:
import plotly.express as px

# Columns based on your request
x_col = "period2" if "period2" in df.columns else "period"
size_col = "n_papers" if "n_papers" in df.columns else None
color_col = "names" 

# Ensure numeric x
df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
# df = df[df[]]
df = df[df[x_col].notna()].copy()

# --- Circular disk layout per x bucket ---
# Parameters to control the circle size in data units
rx = 0.4  # horizontal radius in "year" units
ry = 1.0  # vertical radius in arbitrary y units

def disk_positions(n, rx=0.4, ry=1.0):
    """
    Arrange n points in concentric hex-like rings that fill a disk.
    Returns arrays of x_offset, y_offset centered at (0,0).
    Ring k has capacity 6*k (k>=1), ring 0 has 1 at center.
    """
    xs, ys = [], []
    if n <= 0:
        return np.array([]), np.array([])
    # center
    xs.append(0.0)
    ys.append(0.0)
    placed = 1
    k = 2
    while placed < n:
        cap = 6 * k
        thetas = np.linspace(0, 2*np.pi, cap, endpoint=False)
        for th in thetas:
            if placed >= n:
                break
            xs.append(rx * k * np.cos(th))
            ys.append(ry * k * np.sin(th))
            placed += 1
        k += 1
    return np.array(xs), np.array(ys)

# Within each x group, optionally sort by size descending so large dots gravitate toward center
if size_col and size_col in df.columns:
    df["_size_for_sort"] = df[size_col].fillna(1.0)
else:
    df["_size_for_sort"] = 1.0

df["_x"] = df[x_col]
df["_y"] = 0.0

for xval, idxs in df.groupby(x_col).groups.items():
    ids = list(idxs)
    # sort big -> small
    ids_sorted = df.loc[ids].sort_values("_size_for_sort", ascending=False).index.tolist()
    nx = len(ids_sorted)
    xs_off, ys_off = disk_positions(nx, rx=rx, ry=ry)
    # assign offsets in sorted order
    for i, row_id in enumerate(ids_sorted):
        df.at[row_id, "_x"] = xval + xs_off[i]
        df.at[row_id, "_y"] = ys_off[i]

# --- Plot ---
kwargs = {}
if size_col and size_col in df.columns:
    kwargs.update(size=size_col, size_max=30)

fig = px.scatter(
    df,
    x="_x",
    y="_y",
    color=color_col,
    hover_data=[x_col, color_col] + ([size_col] if size_col in df.columns else []),
    **kwargs
)

# Cosmetics
fig.update_traces(marker=dict(opacity=0.85), selector=dict(mode="markers"))
fig.update_yaxes(showgrid=False, showticklabels=False, zeroline=True, zerolinewidth=1)
fig.update_xaxes(title=x_col)
fig.update_layout(
    title=f"Circular beeswarm per {x_col} (size = {size_col or 'constant'}, color = {color_col})",
    # legend_title=color_col,
    showlegend=False,
    height=560,
    margin=dict(l=40, r=20, t=60, b=40),
)

fig.show()


/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_16757/4194937286.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2000.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[row_id, "_x"] = xval + xs_off[i]


In [22]:
# fig.write_html(f"{path}/results/feature-only-KG/triangle_counts_papers_circular_beeswarm.html", include_plotlyjs="cdn")

In [23]:
color_top10

NameError: name 'color_top10' is not defined

In [ ]:
import plotly.graph_objects as go

df_plt = df[df['n_papers']>0].copy()
df_plt = df_plt.sort_values(['period', 'n_papers'], ascending=[True, False])

for period in df_plt['period'].unique():
    total_weight = df_plt.loc[df_plt['period']==period, 'n_papers'].sum()
    df_plt.loc[df_plt['period']==period, 'normalized_weight'] = df_plt.loc[df_plt['period']==period, 'n_papers'] / total_weight

top10_names = df_plt.sort_values('n_papers', ascending=False).head(10)['names'].unique()
color_top10 = {name: px.colors.qualitative.Dark2[i % len(px.colors.qualitative.Dark2)] for i, name in enumerate(top10_names)}

fig = go.Figure()
fig.add_traces(go.Scatter(
    name = f"Others ({df_plt[~df_plt['names'].isin(top10_names)]['n_papers'].sum()} papers)",
    x= ['-2000', '2001-2005', '2006-2010', '2011-2015', '2016-2020', '2021-2025'],
    y = [1, 1, 1, 1, 1, 1],
    mode = 'lines',
    fill = 'tonexty',
    text = f"Others ({df_plt[~df_plt['names'].isin(top10_names)]['n_papers'].sum()} papers)",
    showlegend=True,
    fillcolor='rgba(180,180,180,0.4)',  # semi-transparent grey
    line=dict(color='rgba(150,150,150,0.3)'),
))


def rgb2rgba(rgb_color, a=0.5):
    rgb = rgb_color.lstrip('rgb(').rstrip(')').split(',')
    r, g, b = [int(c) for c in rgb]
    return f'rgba({r},{g},{b},{a})'


for name in top10_names:
    df_name = df_plt[df_plt['names']==name]
    fig.add_traces(go.Scatter(
        name = name,
        x=df_name["period"],
        y=df_name["normalized_weight"],
        mode='markers',
        stackgroup='one',
        fill='tonexty',
        text=df_name["names"],
        hovertemplate="Year=%{x}<br>Name=%{text}<br>normalized_weight=%{y}<extra></extra>",
        showlegend=True,
        fillcolor=rgb2rgba(color_top10[name], 0.6),
        line=dict(color=rgb2rgba(color_top10[name], 0.8), width=1),
        marker=dict(size=5, color=color_top10[name])
    ))



fig.update_layout(
    showlegend=True,
    template="none",
    # paper_bgcolor = "rgba(0,0,0,0)",
    # plot_bgcolor= "rgba(0,0,0,0)"
)

fig.show()

In [ ]:
fig.write_image(f"{path}/results/feature-only-KG/img/triangle_counts_papers_AreaOnly_legend.png", scale=2, width=800, height=600)
# fig.write_html(f"{path}/results/feature-only-KG/img/triangle_counts_papers_AreaOnly.html", include_plotlyjs="cdn")

# 2. Missing triangles per year

In [ ]:
# --- 1) Clean types and build a canonical tuple key --------------------------
# If your codes are already integers, you can skip the astype lines.
tri = tri.copy()
tri['m.code']  = pd.to_numeric(tri['m.code'], errors='coerce').astype('Int64')
tri['dt.code'] = pd.to_numeric(tri['dt.code'], errors='coerce').astype('Int64')
tri['rq.code'] = pd.to_numeric(tri['rq.code'], errors='coerce').astype('Int64')

# keep only fully-specified triangles
tri_ok = tri.dropna(subset=['m.code','dt.code','rq.code']).copy()
tri_ok['triple'] = list(zip(tri_ok['m.code'].astype(int),
                            tri_ok['dt.code'].astype(int),
                            tri_ok['rq.code'].astype(int)))

# --- 2) Universe of possible combinations -----------------------------------
n_measures = range(1, 9)    
n_dt       = range(1, 15)   
n_rq       = range(1, 10)  

all_possible = {(m, d, r) for m in n_measures for d in n_dt for r in n_rq}
appeared_all = set(tri_ok['triple'])

# --- 3) Focus case: m=1, d=1 -------------------------------------------------
M, D = 1, 2
Rs_all_possible = {r for (m, d, r) in all_possible if m == M and d == D}
Rs_seen_overall = {r for (m, d, r) in appeared_all  if m == M and d == D}

# (A) r's that SHOULD exist (by universe) but NEVER appear in your data at all
missing_overall_rs = sorted(Rs_all_possible - Rs_seen_overall)

missing_overall_df = pd.DataFrame(
    [(M, D, r) for r in missing_overall_rs],
    columns=['m.code','dt.code','rq.code']
)

# --- 4) Period-wise: missing in period A but present in other periods --------
# Assume periods are in tri_ok['period'] (string like '-2000', '2001-2005', …)
# If your column is named differently, change below.
if 'period' not in tri_ok.columns:
    raise KeyError("Expected a 'period' column in tri.")

# r-values per period for m=1,d=1
per = tri_ok.loc[(tri_ok['m.code']==M)&(tri_ok['dt.code']==D), ['rq.code','period']].copy()
per['rq.code'] = per['rq.code'].astype(int)

rs_by_period = per.groupby('period')['rq.code'].agg(lambda s: set(s.tolist())).to_dict()
all_periods  = list(rs_by_period.keys())

rows = []
for A in all_periods:
    rs_in_A      = rs_by_period.get(A, set())
    rs_elsewhere = set().union(*(rs_by_period[p] for p in all_periods if p != A))
    # r that appear somewhere else (same m,d) but NOT in A
    missing_in_A_but_seen_elsewhere = sorted((rs_elsewhere - rs_in_A) & Rs_all_possible)
    # also useful: r that are missing in A relative to the universe (whether or not seen elsewhere)
    missing_in_A_vs_universe        = sorted(Rs_all_possible - rs_in_A)
    rows.append({
        'period': A,
        'missing_in_A_but_seen_elsewhere': missing_in_A_but_seen_elsewhere,
        'missing_in_A_vs_universe': missing_in_A_vs_universe,
        'appeared_in_A': sorted(rs_in_A),
    })

missing_by_period_df = pd.DataFrame(rows).sort_values('period').reset_index(drop=True)

In [ ]:
missing_by_period_df # Focusing on (M,D) = (1,2)

,period,missing_in_A_but_seen_elsewhere,missing_in_A_vs_universe,appeared_in_A
0,-2000,"[1, 2, 3, 5]","[1, 2, 3, 5, 6, 7, 8, 9]",[4]
1,2001-2005,"[1, 3, 4, 5]","[1, 3, 4, 5, 6, 7, 8, 9]",[2]
2,2006-2010,"[3, 5]","[3, 5, 6, 7, 8, 9]","[1, 2, 4]"
3,2011-2015,[5],"[5, 6, 7, 8, 9]","[1, 2, 3, 4]"
4,2016-2020,[],"[6, 7, 8, 9]","[1, 2, 3, 4, 5]"
5,2021-2025,[5],"[5, 6, 7, 8, 9]","[1, 2, 3, 4]"


# 3. Pairs

In [ ]:
pairs = pd.read_csv(f"{path}/results/feature-only-KG/pair_counts_perYear.csv")

In [ ]:
pairs['period2'] = pairs['period'].apply(lambda x: x.split('-')[1].strip())
pairs['names'] = pairs['from_name'] + " <-> " + pairs['to_name']
pairs.head()

,from_label,from_name,reltype,to_label,to_name,period,weight_sum,period2,names
0,Measure,Decomposition / Structural Approaches,CO_MEASURE_DATATYPE,DataType,No dataset,-2000,12,2000,Decomposition / Structural Approaches <-> No d...
1,DataType,No dataset,CO_DATATYPE_RQTYPE,RqType,Theoretical and Structural Models,-2000,10,2000,No dataset <-> Theoretical and Structural Models
2,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Intergenerational Wealth Mobility and Inheritance,-2000,10,2000,Regression‐based Measures <-> Intergenerationa...
3,DataType,Administrative/Registry Data,CO_DATATYPE_RQTYPE,RqType,Intergenerational Wealth Mobility and Inheritance,-2000,8,2000,Administrative/Registry Data <-> Intergenerati...
4,Measure,Decomposition / Structural Approaches,CO_RQTYPE_MEASURE,RqType,Theoretical and Structural Models,-2000,8,2000,Decomposition / Structural Approaches <-> Theo...


In [ ]:
MAP = {**M_MAP, **DT_MAP, **RQ_MAP}
MAP

{'Regression-based Measures': 1,
 'Rank-based Measures': 2,
 'Transition Matrix / Probability Measures': 3,
 'Absolute Mobility Measures': 4,
 'Multigenerational Measures': 5,
 'Decomposition / Structural Approaches': 6,
 'Non-parametric Approaches': 7,
 'Others_Measure': 8,
 'Panel/Longitudinal Surveys': 1,
 'Administrative/Registry Data': 2,
 'National Survey Data': 3,
 'Opportunity Atlas': 4,
 'Natural/Experimental Data': 5,
 'Linked Administrative Data': 6,
 'International Panel Data': 7,
 'Rich List Data': 8,
 'University/Institution Data': 9,
 'Pseudo-Panel/Household Budget Survey': 10,
 'Archival/Historical Data': 11,
 'Big Data': 12,
 'No dataset': 13,
 'Others_DataType': 14,
 'Measurement and Methodological Advances': 1,
 'Empirical Estimates and Determinants': 2,
 'Policy, Institutional, and Geographic Impacts': 3,
 'Intergenerational Wealth Mobility and Inheritance': 4,
 'Demographic Differences in Mobility (Race, Gender, etc.)': 5,
 'Mobility and Non-Income Outcomes (Health

In [ ]:
for i, row in pairs.iterrows():
    if pairs.at[i, 'from_label'] == 'Measure':
        pairs.at[i, 'from_code'] = f"M.{MAP[norm(pairs.at[i, 'from_name'])]}"
    if pairs.at[i, 'to_label'] == 'Measure':
        pairs.at[i, 'to_code'] = f"M.{MAP[norm(pairs.at[i, 'to_name'])]}"
    if pairs.at[i, 'from_label'] == 'DataType':
        pairs.at[i, 'from_code'] = f"DT.{MAP[norm(pairs.at[i, 'from_name'])]}"
    if pairs.at[i, 'to_label'] == 'DataType':
        pairs.at[i, 'to_code'] = f"DT.{MAP[norm(pairs.at[i, 'to_name'])]}"
    if pairs.at[i, 'from_label'] == 'RqType':
        pairs.at[i, 'from_code'] = f"RQ.{MAP[norm(pairs.at[i, 'from_name'])]}"
    if pairs.at[i, 'to_label'] == 'RqType':
        pairs.at[i, 'to_code'] = f"RQ.{MAP[norm(pairs.at[i, 'to_name'])]}"

pairs['code_combined'] = pairs[['from_code', 'to_code']].apply(lambda x: [v for v in x if pd.notna(v)], axis=1)
pairs['code_combined'] = pairs['code_combined'].astype('string')
pairs.head()

,from_label,from_name,reltype,to_label,to_name,period,weight_sum,period2,names,from_code,to_code,code_combined
0,Measure,Decomposition / Structural Approaches,CO_MEASURE_DATATYPE,DataType,No dataset,-2000,12,2000,Decomposition / Structural Approaches <-> No d...,M.6,DT.13,"['M.6', 'DT.13']"
1,DataType,No dataset,CO_DATATYPE_RQTYPE,RqType,Theoretical and Structural Models,-2000,10,2000,No dataset <-> Theoretical and Structural Models,DT.13,RQ.7,"['DT.13', 'RQ.7']"
2,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Intergenerational Wealth Mobility and Inheritance,-2000,10,2000,Regression‐based Measures <-> Intergenerationa...,M.1,RQ.4,"['M.1', 'RQ.4']"
3,DataType,Administrative/Registry Data,CO_DATATYPE_RQTYPE,RqType,Intergenerational Wealth Mobility and Inheritance,-2000,8,2000,Administrative/Registry Data <-> Intergenerati...,DT.2,RQ.4,"['DT.2', 'RQ.4']"
4,Measure,Decomposition / Structural Approaches,CO_RQTYPE_MEASURE,RqType,Theoretical and Structural Models,-2000,8,2000,Decomposition / Structural Approaches <-> Theo...,M.6,RQ.7,"['M.6', 'RQ.7']"


In [ ]:
# pairs.to_csv(f"{path}/results/feature-only-KG/pair_counts_perYear_p2.csv", index=False)

In [ ]:
pairs.groupby('period2').apply(lambda g: g.nlargest(3, 'weight_sum')).reset_index(drop=True)

/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_68348/2899285607.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,from_label,from_name,reltype,to_label,to_name,period,weight_sum,period2,names,from_code,to_code,code_combined
0,Measure,Decomposition / Structural Approaches,CO_MEASURE_DATATYPE,DataType,No dataset,-2000,12,2000,Decomposition / Structural Approaches <-> No d...,M.6,DT.13,"['M.6', 'DT.13']"
1,DataType,No dataset,CO_DATATYPE_RQTYPE,RqType,Theoretical and Structural Models,-2000,10,2000,No dataset <-> Theoretical and Structural Models,DT.13,RQ.7,"['DT.13', 'RQ.7']"
2,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Intergenerational Wealth Mobility and Inheritance,-2000,10,2000,Regression‐based Measures <-> Intergenerationa...,M.1,RQ.4,"['M.1', 'RQ.4']"
3,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Empirical Estimates and Determinants,2001-2005,14,2005,Regression‐based Measures <-> Empirical Estima...,M.1,RQ.2,"['M.1', 'RQ.2']"
4,Measure,Regression‐based Measures,CO_MEASURE_DATATYPE,DataType,Panel/Longitudinal Surveys,2001-2005,10,2005,Regression‐based Measures <-> Panel/Longitudin...,M.1,DT.1,"['M.1', 'DT.1']"
5,DataType,Panel/Longitudinal Surveys,CO_DATATYPE_RQTYPE,RqType,Empirical Estimates and Determinants,2001-2005,8,2005,Panel/Longitudinal Surveys <-> Empirical Estim...,DT.1,RQ.2,"['DT.1', 'RQ.2']"
6,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Intergenerational Wealth Mobility and Inheritance,2006-2010,46,2010,Regression‐based Measures <-> Intergenerationa...,M.1,RQ.4,"['M.1', 'RQ.4']"
7,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Empirical Estimates and Determinants,2006-2010,32,2010,Regression‐based Measures <-> Empirical Estima...,M.1,RQ.2,"['M.1', 'RQ.2']"
8,Measure,Regression‐based Measures,CO_MEASURE_DATATYPE,DataType,Panel/Longitudinal Surveys,2006-2010,30,2010,Regression‐based Measures <-> Panel/Longitudin...,M.1,DT.1,"['M.1', 'DT.1']"
9,Measure,Regression‐based Measures,CO_RQTYPE_MEASURE,RqType,Intergenerational Wealth Mobility and Inheritance,2011-2015,84,2015,Regression‐based Measures <-> Intergenerationa...,M.1,RQ.4,"['M.1', 'RQ.4']"


In [ ]:
import plotly.graph_objects as go

pairs_plt = pairs[pairs['weight_sum']>0].copy()
pairs_plt = pairs_plt.sort_values(['period', 'weight_sum'], ascending=[True, False])
for period in pairs_plt['period'].unique():
    total_weight = pairs_plt[pairs_plt['period']==period]['weight_sum'].sum()
    pairs_plt.loc[pairs_plt['period']==period, 'normalized_weight'] = pairs_plt.loc[pairs_plt['period']==period, 'weight_sum'] / total_weight


fig = go.Figure()


fig.add_traces(go.Scatter(
    name = "Others",
    x= ['-2000', '2001-2005', '2006-2010', '2011-2015', '2016-2020', '2021-2025'],
    y = [1, 1, 1, 1, 1, 1],
    mode = 'lines',
    fill = 'tonexty',
    text = "Others",
    showlegend=True,
    fillcolor='rgba(180,180,180,0.4)',  # semi-transparent grey
    line=dict(color='rgba(180,180,180,0.3)')
))



top10_names = pairs_plt.sort_values('normalized_weight', ascending=False).head(10)['names'].unique()
color_top10_pairs = {name: px.colors.qualitative.Dark2[i % len(px.colors.qualitative.Dark2)] for i, name in enumerate(top10_names)}

for name in top10_names:
# for name in pairs_plt['names'].unique():
    df_name = pairs_plt[pairs_plt['names']==name]
    fig.add_traces(go.Scatter(
        name = name,
        x=df_name["period"],
        y=df_name["normalized_weight"],
        mode='markers',
        stackgroup='one',
        fill='tonexty',
        text=df_name["names"],
        hovertemplate="Year=%{x}<br>Name=%{text}<br>normalized_weight=%{y}<extra></extra>",
        showlegend=True,
        fillcolor=rgb2rgba(color_top10_pairs[name], 0.6),
        line=dict(color=rgb2rgba(color_top10_pairs[name], 0.8), width=1),
        marker=dict(size=5, color=color_top10_pairs[name])
    ))
    fig.update_layout(legend_title_text='Top pairs', )




fig.update_layout(
    showlegend=True,
    template = "none",
    # plot_bgcolor='rgba(0,0,0,0)',
    # paper_bgcolor = 'rgba(0,0,0,0)',
)

fig.show()

In [ ]:
# fig.write_html(f"{path}/results/feature-only-KG/img/pair_counts_perYear_AreaOnly.html", include_plotlyjs="cdn")
fig.write_image(f"{path}/results/feature-only-KG/img/pair_counts_perYear_AreaOnly_legend.png", scale=2, width=800, height=600)

# 4. Nodes

In [ ]:
top_degree_all = {}
for file in os.listdir(f"{path}/results/feature-only-KG/periods/top_degree"):
    if file.endswith(".csv"):
        period = file.split("_")[0]
        df = pd.read_csv(f"{path}/results/feature-only-KG/periods/top_degree/{file}")
        df['period'] = period
        top_degree_all[period] = df
all_top_degree_df = pd.concat(top_degree_all.values(), ignore_index=True)
all_top_degree_df.head()

,Unnamed: 0,kind,node,score,period
0,0,Measure,Regression‐based Measures,16.0,2016-2020
1,1,RqType,Intergenerational Wealth Mobility and Inheritance,15.0,2016-2020
2,2,RqType,Empirical Estimates and Determinants,15.0,2016-2020
3,3,Measure,Others_Measure,14.0,2016-2020
4,4,Measure,Absolute Mobility Measures,13.0,2016-2020


In [ ]:
import plotly.express as px
import plotly.graph_objects as go   

period_order = ["-2000", "2001-2005", "2006-2010", "2011-2015", "2016-2020", "2021-2025"]

plt_df = all_top_degree_df[all_top_degree_df['score']>0].copy()
plt_df['count_appearnce'] = plt_df.groupby('node').cumcount() + 1
plt_df = plt_df.sort_values(['period', 'score'], ascending=[True, False])

fig = px.line(
    plt_df,
    x='period',
    y='score',
    color='node',
    title='Node Appearances Over Time',
    labels={'count_appearance': 'Number of Appearances', 'period': 'Time Period'},
    category_orders={'period': period_order}
)

fig.show()

# 5. Resurgence Analysis

## [Node-approach] (Paper-level)

In [2]:
df = pd.read_csv(f'{path}/data_abstracts/true_mobility_studies_617_forKGs_cleaned.csv')
df = df[['year', 'category_1', 'data_cat', 'rq_cat']].copy()
print(len(df))
df.head()

617


,year,category_1,data_cat,rq_cat
0,2017,Absolute Mobility Measures,Linked Administrative Data,Intergenerational Wealth Mobility and Inheritance
1,1998,Regression‐based Measures,No dataset,Empirical Estimates and Determinants
2,1980,Regression‐based Measures,No dataset,Empirical Estimates and Determinants
3,2009,Transition Matrix / Probability Measures,Panel/Longitudinal Surveys,Measurement and Methodological Advances
4,2020,Absolute Mobility Measures,Others,Empirical Estimates and Determinants


In [3]:
df['year'].min(), df['year'].max()

(1976, 2025)

In [4]:
replacements = {
    'category_1': {'Others': 'Others_Measure'},
    'data_cat':   {'Others': 'Others_DataType'},
    'rq_cat':     {'Others': 'Others_RqType'},
}
df_renamed = df.replace(replacements)

# 2) Make it long
long_df = df_renamed.melt(
    id_vars='year',
    value_vars=['category_1', 'data_cat', 'rq_cat'],
    var_name='type',
    value_name='category'
)

# (Optional) nicer labels for the dimension column
type_map = {
    'category_1': 'Measure',
    'data_cat':   'DataType',
    'rq_cat':     'RqType'
}
long_df['type'] = long_df['type'].map(type_map)
long_df['identifier'] = "appeared"


In [5]:
long_df = long_df.groupby(['year', 'type', 'category', 'identifier']).size().reset_index(name='paper_count')
long_df.head()

,year,type,category,identifier,paper_count
0,1976,DataType,Archival/Historical Data,appeared,1
1,1976,Measure,Absolute Mobility Measures,appeared,1
2,1976,RqType,Intergenerational Wealth Mobility and Inheritance,appeared,1
3,1979,DataType,Administrative/Registry Data,appeared,1
4,1979,DataType,No dataset,appeared,2


In [8]:
import decay_weight
import math
df_years_filled = decay_weight.fill_missing_years(long_df, year_col='year', key_col = 'category',  count_col='paper_count', identifier_col='identifier',key2_col = None, is_triplet=False)
df_years_filled = decay_weight.compute_decaying_weights(df_years_filled, lam=math.log(2)/5, year_col='year', key_col = 'key', count_col='paper_count', identifier_col='identifier', is_weighted=True)

1 0.8705505632961241
0.8705505632961241 0.8705505632961241
0.757858283255199 0.8705505632961241
0.6597539553864471 0.8705505632961241
0.5743491774985174 0.8705505632961241
0.49999999999999994 0.8705505632961241
0.435275281648062 0.8705505632961241
0.37892914162759944 0.8705505632961241
0.3298769776932235 0.8705505632961241
0.28717458874925866 0.8705505632961241
1.2871745887492587 0.8705505632961241
1.1205505632961241 0.8705505632961241
0.97549592407923 0.8705505632961241
0.8492185262002468 0.8705505632961241
0.7392876663451292 0.8705505632961241
0.6435872943746292 0.8705505632961241
0.560275281648062 0.8705505632961241
0.4877479620396149 0.8705505632961241
0.4246092631001233 0.8705505632961241
0.3696438331725645 0.8705505632961241
0.32179364718731457 0.8705505632961241
0.2801376408240309 0.8705505632961241
0.24387398101980742 0.8705505632961241
0.21230463155006163 0.8705505632961241
0.18482191658628225 0.8705505632961241
0.16089682359365728 0.8705505632961241
1.1608968235936572 0.87055

In [14]:
decay = np.exp(-np.log(2)/5)
val = 1.0   
for i in range(20):
    val = val * decay + 1
    print(f"after iter {i+1}: {val}")
print("final result:", val)

after iter 1: 1.8705505632961241
after iter 2: 2.628408846551323
after iter 3: 3.28816280193777
after iter 4: 3.8625119794362877
after iter 5: 4.362511979436288
after iter 6: 4.79778726108435
after iter 7: 5.17671640271195
after iter 8: 5.506593380405174
after iter 9: 5.793767969154432
after iter 10: 6.043767969154432
after iter 11: 6.261405609978463
after iter 12: 6.450870180792263
after iter 13: 6.6158086696388745
after iter 14: 6.759395964013504
after iter 15: 6.884395964013504
after iter 16: 6.993214784425519
after iter 17: 7.087947069832419
after iter 18: 7.170416314255725
after iter 19: 7.24220996144304
after iter 20: 7.30470996144304
final result: 7.30470996144304


In [9]:
df_years_filled.to_csv(f"{path}/results/feature-only-KG/decaying_weights_over_time_node_approach_papercounts_allyears2.csv", index=False)

## [Edge-approach]

In [10]:
import graph
import decay_weight

df = pd.read_csv(f"{path}/results/feature-only-KG/edges_appearances_paperids.csv")
df = df[['m.name', 'n.name', 'year', 'paper_count']]
df['m.name'] = df['m.name'].apply(graph.norm)
df['n.name'] = df['n.name'].apply(graph.norm)
df['year'] = df['year'].apply(decay_weight.parse_list_like)
df = df.explode("year").reset_index(drop=True)
df['identifier'] = "appeared"
df

,m.name,n.name,year,paper_count,identifier
0,Absolute Mobility Measures,Archival/Historical Data,1976,2,appeared
1,Absolute Mobility Measures,Archival/Historical Data,2004,2,appeared
2,Absolute Mobility Measures,No dataset,2010,6,appeared
3,Absolute Mobility Measures,No dataset,2013,6,appeared
4,Absolute Mobility Measures,No dataset,2014,6,appeared
...,...,...,...,...,...
1985,University/Institution Data,Empirical Estimates and Determinants,2019,2,appeared
1986,Big Data,Absolute Mobility Measures,2022,2,appeared
1987,Big Data,Regression-based Measures,2025,1,appeared
1988,Big Data,Empirical Estimates and Determinants,2022,3,appeared


In [13]:
import math

df_years_filled = decay_weight.fill_missing_years(df, year_col='year', key_col = 'm.name',  count_col='paper_count', identifier_col='identifier', key2_col = "n.name", is_triplet=False)
df_years_filled = decay_weight.compute_decaying_weights(df_years_filled, lam=math.log(2)/5, year_col='year', key_col = 'key1', key2_col='key2', count_col='paper_count', identifier_col='identifier', is_weighted=True)

In [14]:
df_years_filled.to_csv(f"{path}/results/feature-only-KG/decaying_weights_over_time_edge_approach_papercounts_allyears2.csv", index=False)

## [Triplet-approach]: Weighted but the result is the same as unweighted because of non repeating triplet at each year

In [15]:
import pandas as pd
import decay_weight

df = pd.read_csv(f"{path}/results/feature-only-KG/triplet_appearances_paperids.csv")

# Apply to both columns
df["paper_ids"] = df["paper_ids"].apply(decay_weight.parse_list_like)
df["years"] = df["years"].apply(decay_weight.parse_list_like)
df["ew"] = df["ew"].apply(decay_weight.parse_list_like)

rows = []
for _, row in df.iterrows():
    papers = row["paper_ids"]
    years = row["years"]
    ews = row["ew"]

    # align each paper ID with its year
    for y, pid, ew in zip(years, papers, ews):
        new_row = row.to_dict()
        new_row["years"] = y
        new_row["paper_ids"] = pid
        new_row["ew"] = ew
        rows.append(new_row)

df_expanded = pd.DataFrame(rows)

df_expanded = df_expanded.rename(columns={'years': 'year'})
df_summarized = df_expanded.groupby(['triplet_key', 'year', 'measure', 'datatype', 'rqtype']).agg({
    'paper_ids': 'count'}).reset_index().rename(columns={'paper_ids': 'paper_count'})
df_summarized['identifier'] = 'appeared'


print(len(df), len(df_expanded), len(df_summarized))
df_summarized.head()


158 412 412


,triplet_key,year,measure,datatype,rqtype,paper_count,identifier
0,Absolute Mobility Measures | Administrative/Re...,2024,Absolute Mobility Measures,Administrative/Registry Data,Empirical Estimates and Determinants,1,appeared
1,Absolute Mobility Measures | Administrative/Re...,2014,Absolute Mobility Measures,Administrative/Registry Data,Intergenerational Wealth Mobility and Inheritance,1,appeared
2,Absolute Mobility Measures | Archival/Historic...,1976,Absolute Mobility Measures,Archival/Historical Data,Intergenerational Wealth Mobility and Inheritance,1,appeared
3,Absolute Mobility Measures | Big Data | Empiri...,2022,Absolute Mobility Measures,Big Data,Empirical Estimates and Determinants,1,appeared
4,Absolute Mobility Measures | Linked Administra...,2021,Absolute Mobility Measures,Linked Administrative Data,Empirical Estimates and Determinants,1,appeared


In [16]:
import decay_weight
import math

df_years_filled = decay_weight.fill_missing_years(df_summarized, year_col='year', key_col = 'triplet_key',  count_col='paper_count', identifier_col='identifier',key2_col = None, is_triplet=True)
df_years_filled = decay_weight.compute_decaying_weights(df_years_filled, lam=math.log(2)/5, year_col='year', key_col = 'key', count_col='paper_count', identifier_col='identifier', is_weighted=False)

0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.8705505632961241
0.0 0.87055

In [18]:
df_years_filled.to_csv(f"{path}/results/feature-only-KG/decaying_weights_over_time_triplet_approach_papercounts_allyears2.csv", index=False)